### Import Dependencies

In [1]:
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance, PayloadSchemaType, PointStruct, SparseVectorParams, Document, Prefetch, FusionQuery
from qdrant_client import models

import pandas as pd
import openai
import cohere

### Retrieval

In [2]:
qdrant_client = QdrantClient(url="http://localhost:6333")

In [3]:
def get_embedding(text, model="text-embedding-3-small"):
    response = openai.embeddings.create(
        input=[text],
        model=model,
    )
    return response.data[0].embedding

In [4]:
def retrieve_data(query, qdrant_client, k=5):

    query_embedding = get_embedding(query)

    results = qdrant_client.query_points(
        collection_name="Amazon-items-collection-01-hybrid-search",
        prefetch=[
            Prefetch(
                query=query_embedding,
                using="text-embedding-3-small",
                limit=20
            ),
            Prefetch(
                query=Document(
                    text=query,
                    model="qdrant/bm25"
                ),
                using="bm25",
                limit=20
            )
        ],
        query=FusionQuery(fusion="rrf"),
        limit=k,
    )

    retrieved_context_ids = []
    retrieved_context = []
    similarity_scores = []
    retrieved_context_ratings = []

    for result in results.points:
        retrieved_context_ids.append(result.payload["parent_asin"])
        retrieved_context.append(result.payload["description"])
        retrieved_context_ratings.append(result.payload["average_rating"])
        similarity_scores.append(result.score)

    return {
        "retrieved_context_ids": retrieved_context_ids,
        "retrieved_context": retrieved_context,
        "retrieved_context_ratings": retrieved_context_ratings,
        "similarity_scores": similarity_scores,
    }

In [5]:
query = "Can I get a laptop?"

In [6]:
results = retrieve_data(query, qdrant_client, k=20)

In [7]:
results

{'retrieved_context_ids': ['B0BGSHMC98',
  'B0B18FLDJB',
  'B0C33ZB3H6',
  'B0B476B13X',
  'B0C74FK8R4',
  'B0C7CG4CNS',
  'B0C4PG4MQY',
  'B0B27RZKW8',
  'B0C1Z94D74',
  'B0B5FZGQHH',
  'B0B3R22F1J',
  'B0C4H2BK5C',
  'B0BZRD34C2',
  'B09SYXDD15',
  'B0BJSZ9KRG',
  'B09ZKLVMXF',
  'B0B12N643H',
  'B0BKTKFSQW',
  'B0B63LX7WX',
  'B09XMPJG2W'],
 'retrieved_context': ["Wireless Game Controller for Windows PC/Raspberry Pi/PS3 Gamepad Rechargeable Gaming Joystick Portable Remote Joypad for Laptop/Desktop Computer,Emulator,RetroArch, Steam,Roblox,RetroPie,RecalBox [Wireless Controller with Powerful Compatibility] - Support not only for Laptop/Desktop PC(Windows 11/10/8/7/XP), PS3, Raspberry Pi(4/3/2/1/Zero), RetroPie, RecalBox, Steam, Roblox, Emulator, RetroArch, but also for Android devices(Support OTG Function), such as Projector, Smart TV, TV/Game Box, Tesla, Phone/Tablet/Pad(Work with OTG Converter, Not Included), Amazon Fire TV/TV Stick(Work with USB OTG Adapter, Not Included). doesn't

### Reranking

In [8]:
cohere_client = cohere.ClientV2()

In [9]:
to_rerank = results["retrieved_context"]

In [10]:
to_rerank

["Wireless Game Controller for Windows PC/Raspberry Pi/PS3 Gamepad Rechargeable Gaming Joystick Portable Remote Joypad for Laptop/Desktop Computer,Emulator,RetroArch, Steam,Roblox,RetroPie,RecalBox [Wireless Controller with Powerful Compatibility] - Support not only for Laptop/Desktop PC(Windows 11/10/8/7/XP), PS3, Raspberry Pi(4/3/2/1/Zero), RetroPie, RecalBox, Steam, Roblox, Emulator, RetroArch, but also for Android devices(Support OTG Function), such as Projector, Smart TV, TV/Game Box, Tesla, Phone/Tablet/Pad(Work with OTG Converter, Not Included), Amazon Fire TV/TV Stick(Work with USB OTG Adapter, Not Included). doesn't support for Xbox 360/Xbox One/iOS/Mac OS/PS4. [Rechargeable Wireless PC Gamepad] - High performance built-in lithium battery, no need to remove and replace battery, safe and environmentally friendly, saving your money to buy batteries. intelligent sleep and wake-up functions effectively extend the battery life. charging time: 2 hours, gaming time: 4+ hours, standby

In [11]:
response = cohere_client.rerank(
    model="rerank-v4.0-pro",
    query=query,
    documents=to_rerank,
    top_n=20
)

In [12]:
response

V2RerankResponse(id='2a9b6b0c-c95c-4b6c-b3e0-5306fed2d502', results=[V2RerankResponseResultsItem(index=5, relevance_score=0.69562757), V2RerankResponseResultsItem(index=7, relevance_score=0.69562757), V2RerankResponseResultsItem(index=8, relevance_score=0.69562757), V2RerankResponseResultsItem(index=10, relevance_score=0.67542213), V2RerankResponseResultsItem(index=12, relevance_score=0.67542213), V2RerankResponseResultsItem(index=11, relevance_score=0.6685341), V2RerankResponseResultsItem(index=18, relevance_score=0.66506255), V2RerankResponseResultsItem(index=9, relevance_score=0.66332006), V2RerankResponseResultsItem(index=0, relevance_score=0.6615731), V2RerankResponseResultsItem(index=13, relevance_score=0.658066), V2RerankResponseResultsItem(index=6, relevance_score=0.6563059), V2RerankResponseResultsItem(index=1, relevance_score=0.65277284), V2RerankResponseResultsItem(index=17, relevance_score=0.6492228), V2RerankResponseResultsItem(index=15, relevance_score=0.64744157), V2Rera

In [13]:
reranked_results = [to_rerank[result.index] for result in response.results]

In [14]:
reranked_results

['WolfLawS TA83 Wireless Gaming Headset for PS5 PS4 PC, 2.4G Bluetooth USB Gamer Headphones with Noise Canceling Mic for Mac Laptop Computer Switch Mobile Retractable Microphone & Easy-acccess Controls - The flexible microphone not only can rotate in all the way around that you can get it at a prefect distance from your mouth, but also can retract inside the headset while you don’t use it. The one key mute and volume control on the earphones make it easily adjust volume or mute microphone when you are in games. 2.4GHz Wireless Connection & Wired Mode -- TA83 wireless headphone could be perfectly compatible with PC, PS4, PS5, Laptop and Mac. And please note that if you want to connect the headset with the Xbox devices, you need to plug the 3.5mm cable into the headset, only the wired mode could be compatible with Xbox one, Xbox Series, Nintendo Switch, Mobile phones or any devices with 3.5mm port. Headset also has the Bluetooth Mode to support the devices which get the BT function. 50mm

In [15]:
response = cohere_client.rerank(
    model="rerank-v4.0-fast",
    query=query,
    documents=to_rerank,
    top_n=20
)

In [16]:
reranked_results = [to_rerank[result.index] for result in response.results]

In [17]:
reranked_results

["TECURS 2.4G Wireless Keyboard Ultra Slim Full Size Keyboard with Numeric Keypad, Rechargeable LED Backlit Mac Keyboard Silent Keyboard with Multimedia Keys for Mac and Windows 【Easy To Use】Advanced RF 2.4G wireless technology that delivers anti-interference and reliable connection.with USB receiver ( no driver needed ) plug and play. enjoy up to 10m/33ft operating distance for PC, Windows and Mac OS 【8 Multimedia Hotkeys Wireless Keyboard】8 Multimedia hotkeys easily access to Setup, Homepage, Music, Volume, Internet, E-mail and Calculator, let you quickly and easily achieve the operation you want. 【Super Quiet, No Latency】The silent design of this ergonomic keyboard, boasts a thin-film membrane structure that offers slip-proof keys. It adds stability, quick press, and bounce for enhanced comfort. It reduces typing noise by over 95% while providing typing comfort. Shows your love and care about your surroundings, especially in the library, coffee shop, or late work at night. 【Ergonomi